In [1]:
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

from PIL import Image
import numpy as np
import sys
import math
import os

import torch
from torch import nn
from torch.autograd import Variable, Function
from torchvision import models, transforms
import torchvision.models as models

%matplotlib inline

print("mpl backend: ", plt.get_backend())

mpl backend:  module://ipykernel.pylab.backend_inline


In [2]:
torch.cuda.set_device(3)

In [3]:
classifier_size = 512 * 2 * 2

model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(classifier_size, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'LITTLE': [32, 'M', 64, 'M', 128, 128, 'M', 256, 256, 'M', 256, 256, 'M']
}


def vgg11(pretrained=False, **kwargs):
    """VGG 11-layer model (configuration "A")
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfg['A']), **kwargs)
    if pretrained:
        model.load_state_dict( model_zoo.load_url(model_urls['vgg11']))
    return model


def vgg11_little(pretrained=False, **kwargs):
    """VGG 11-layer model (configuration "A") with batch normalization
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfg['A'], batch_norm=False), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
    return model

In [4]:
vgg_pretrained = vgg11_little(num_classes=200)
vgg_pretrained = vgg_pretrained.cuda()
resume='model_best.pth.tar'
if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    checkpoint = torch.load(resume)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    vgg_pretrained.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})"
          .format(resume, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))

=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 2)


In [5]:
classifier_size = 512 * 2 * 2

class vgg_conv(torch.nn.Module):

    def __init__(self, features, m_classes=1000):
        super(vgg_conv, self).__init__()
        # VGG16 (using return_indices=True on the MaxPool2d layers)
        self.features = torch.nn.Sequential(
            # conv1
            torch.nn.Conv2d(3, 64, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv2
            torch.nn.Conv2d(64, 128, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv3
            torch.nn.Conv2d(128, 256, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv4
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv5
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True))
        self.feature_outputs = [0]*len(self.features)
        self.pool_indices = dict()

        self.classifier = vgg_pretrained.classifier
        self._initialize_weights()


    def _initialize_weights(self):
        # initializing weights using ImageNet-trained model from PyTorch
        for i, layer in enumerate(vgg_pretrained.features):
            print("initializing layer {}: {}".format(i, layer))
            try:
                self.features[i].weight.data = layer.weight.data
                self.features[i].bias.data = layer.bias.data
            except:
                continue

    def get_conv_layer_indices(self):
        return [0, 3, 6, 8, 11, 13, 16, 18]

    def forward_features(self, x):
        output = x
        for i, layer in enumerate(self.features):
            if isinstance(layer, torch.nn.MaxPool2d):
                output, indices = layer(output)
                self.feature_outputs[i] = output
                self.pool_indices[i] = indices
            else:
                output = layer(output)
                self.feature_outputs[i] = output
        return output

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [6]:
class vgg_deconv(torch.nn.Module):
    def __init__(self):
        super(vgg_deconv, self).__init__()
        self.relu = torch.nn.ReLU()
        
        self.conv2DeconvIdx = {0:12, 3:10, 6:8, 8:7, 11:5, 13:4, 16:2, 18:1}
        self.conv2DeconvBiasIdx = {0:10, 3:8, 6:7, 8:5, 11:4, 13:2, 16:1, 18:0}
        self.unpool2PoolIdx = {0:20, 3:15, 6:10, 9:5, 11:2}
        
        self.deconv_features = torch.nn.Sequential(
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(512, 512, 3, padding=1),
            torch.nn.ConvTranspose2d(512, 512, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(512, 512, 3, padding=1),
            torch.nn.ConvTranspose2d(512, 512, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(256, 256, 3, padding=1),
            torch.nn.ConvTranspose2d(256, 128, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(128, 64, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(64, 3, 3, padding=1))

        # not the most elegant, given that I don't need the MaxUnpools here
        self.deconv_first_layers = torch.nn.ModuleList([
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(1, 512, 3, padding=1),
            torch.nn.ConvTranspose2d(1, 512, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(1, 512, 3, padding=1),
            torch.nn.ConvTranspose2d(1, 512, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(1, 256, 3, padding=1),
            torch.nn.ConvTranspose2d(1, 128, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(1, 64, 3, padding=1),
            torch.nn.MaxUnpool2d(2, stride=2),
            torch.nn.ConvTranspose2d(1, 3, 3, padding=1)])

        self._initialize_weights()

    def _initialize_weights(self):
        # initializing weights using ImageNet-trained model from PyTorch
        for i, layer in enumerate(vgg_pretrained.features):
            if isinstance(layer, torch.nn.Conv2d):
                self.deconv_features[self.conv2DeconvIdx[i]].weight.data = layer.weight.data
                biasIdx = self.conv2DeconvBiasIdx[i]
                if biasIdx > 0:
                    self.deconv_features[biasIdx].bias.data = layer.bias.data
                

    def forward(self, x, layer_number, map_number, pool_indices):
        start_idx = self.conv2DeconvIdx[layer_number]
        if not isinstance(self.deconv_first_layers[start_idx], torch.nn.ConvTranspose2d):
            raise ValueError('Layer '+str(layer_number)+' is not of type Conv2d')
        # set weight and bias
        self.deconv_first_layers[start_idx].weight.data = self.deconv_features[start_idx].weight[map_number].data[None, :, :, :]
        self.deconv_first_layers[start_idx].bias.data = self.deconv_features[start_idx].bias.data        
        # first layer will be single channeled, since we're picking a particular filter
        output = self.deconv_first_layers[start_idx](x)

        # transpose conv through the rest of the network
        for i in range(start_idx+1, len(self.deconv_features)):
            if isinstance(self.deconv_features[i], torch.nn.MaxUnpool2d):
                output = self.deconv_features[i](output, pool_indices[self.unpool2PoolIdx[i]])
            else:
                output = self.deconv_features[i](output)
                if i != len(self.deconv_features)-1:
                    output = self.relu(output)
        return output

In [7]:
vgg_c = vgg_conv(200)

initializing layer 0: Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
initializing layer 1: ReLU(inplace)
initializing layer 2: MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
initializing layer 3: Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
initializing layer 4: ReLU(inplace)
initializing layer 5: MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
initializing layer 6: Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
initializing layer 7: ReLU(inplace)
initializing layer 8: Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
initializing layer 9: ReLU(inplace)
initializing layer 10: MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
initializing layer 11: Conv2d (256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
initializing layer 12: ReLU(inplace)
initializing layer 13: Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
initializing layer 14:

In [9]:
def it(layers, features):
    for i in layers:
        yield i, features[i]
def enumerate_shift(shift, iterable):
    for i, v in enumerate(iterable):
        if i >= shift:
            yield i, v
    

In [14]:
vgg_pretrained

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (11): Conv2d (256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (16): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [59]:
layers_to_process = [0, 3, 6]
n_filters = {0: 64, 3: 128, 6: 256}

keep_filter_indices = {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 13, 14, 15, 17, 18, 19, 21, 23, 24, 26, 28, 29, 33, 36, 44, 47, 49, 57], 
                       3:[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 87, 89, 91, 92, 94, 96, 98, 99, 100, 101, 102, 103, 104, 106, 108, 109, 110, 112, 113, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127], 
                       6: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 68, 70, 71, 72, 74, 76, 77, 78, 79, 82, 83, 88, 90, 92, 93, 96, 99, 100, 101, 102, 103, 104, 105, 106, 108, 109, 112, 113, 114, 115, 118, 119, 121, 122, 123, 126, 127, 128, 129, 130, 131, 133, 134, 137, 138, 140, 141, 142, 143, 145, 147, 148, 151, 153, 154, 155, 156, 159, 160, 162, 163, 167, 172, 173, 174, 175, 176, 177, 178, 180, 182, 187, 188, 190, 192, 199, 201, 202, 205, 206, 208, 211, 212, 214, 221, 239, 241, 250, 253, 254, 255]}
remove_filter_indices = {layer_i: [j for j in range(n_filters[layer_i]) if not j in keep_i] for layer_i, keep_i in keep_filter_indices.items()}
#remove_filter_indices = {0: [1,2,3], 3: [1,2,3], 6: [1,2,3]}

print(remove_filter_indices)
remove_num_filt = {k:len(v) for k, v in remove_filter_indices.items()}
filt_f = lambda l: lambda x: not x in l
indices = {k: list(filter(filt_f(remove_filter_indices[k]), range(v.weight.data.shape[0]))) for k, v in it(layers_to_process, vgg_pretrained.features)}
indices_in = {k: list(filter(filt_f(remove_filter_indices[layers_to_process[i-1]]), range(v.weight.data.shape[1]))) for i, (k, v) in enumerate_shift(1, it(layers_to_process, vgg_pretrained.features))}
indices_in[0] = [0, 1, 2]
indices_in[8] = indices[6]
last_layer = 256
indices[8] = list(range(last_layer))
for i, j in indices_in.items():
    #print(i, remove_num_filt[i], len(indices[i]), len(indices_in[i]))
    print(i, indices_in[i])
    print()

{0: [9, 10, 12, 16, 20, 22, 25, 27, 30, 31, 32, 34, 35, 37, 38, 39, 40, 41, 42, 43, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63], 3: [4, 14, 15, 20, 25, 28, 40, 41, 42, 47, 55, 61, 62, 65, 81, 83, 84, 85, 86, 88, 90, 93, 95, 97, 105, 107, 111, 114, 115], 6: [24, 26, 42, 45, 53, 64, 67, 69, 73, 75, 80, 81, 84, 85, 86, 87, 89, 91, 94, 95, 97, 98, 107, 110, 111, 116, 117, 120, 124, 125, 132, 135, 136, 139, 144, 146, 149, 150, 152, 157, 158, 161, 164, 165, 166, 168, 169, 170, 171, 179, 181, 183, 184, 185, 186, 189, 191, 193, 194, 195, 196, 197, 198, 200, 203, 204, 207, 209, 210, 213, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 240, 242, 243, 244, 245, 246, 247, 248, 249, 251, 252]}
3 [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 13, 14, 15, 17, 18, 19, 21, 23, 24, 26, 28, 29, 33, 36, 44, 47, 49, 57]

6 [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36

In [60]:
class VGG16_conv_rem(torch.nn.Module):
    def __init__(self, n_classes):
        super(VGG16_conv_rem, self).__init__()
        # VGG16 (using return_indices=True on the MaxPool2d layers)
        self.features = torch.nn.Sequential(
            # conv1
            torch.nn.Conv2d(len(indices_in[0]), len(indices[0]), 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv2
            torch.nn.Conv2d(len(indices_in[3]), len(indices[3]), 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv3
            torch.nn.Conv2d(len(indices_in[6]), len(indices[6]), 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(len(indices[6]), 256, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv4
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True),
            # conv5
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, stride=2, return_indices=True))
        self.feature_outputs = [0]*len(self.features)
        self.pool_indices = dict()

        self.classifier = vgg_pretrained.classifier
        self._initialize_weights()


    def _initialize_weights(self):
        # initializing weights using ImageNet-trained model from PyTorch
        for i, layer in enumerate(vgg_pretrained.features):
            print("initializing layer {}: {}".format(i, layer))
            try:
                if i in layers_to_process or i == 8:
                    print("        HERE1: ", i, self.features[i].weight.data.shape)
                    print("        HERE2: ", i, layer.weight.data[indices[i],][:,indices_in[i],].shape)
                    
                    self.features[i].weight.data = layer.weight.data[indices[i],][:,indices_in[i],]
                    self.features[i].bias.data = layer.bias.data[indices[i],]
                else:
                    self.features[i].weight.data = layer.weight.data
                    self.features[i].bias.data = layer.bias.data
            except Exception as e:
                print("    E: ", i, e)
                continue

    def get_conv_layer_indices(self):
        return [0, 3, 6, 8, 11, 13, 16, 18]

    def forward_features(self, x):
        output = x
        for i, layer in enumerate(self.features):
            if isinstance(layer, torch.nn.MaxPool2d):
                output, indices = layer(output)
                self.feature_outputs[i] = output
                self.pool_indices[i] = indices
            else:
                output = layer(output)
                self.feature_outputs[i] = output
        return output

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [61]:
x = VGG16_conv_rem(200)

initializing layer 0: Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        HERE1:  0 torch.Size([28, 3, 3, 3])
        HERE2:  0 torch.Size([28, 3, 3, 3])
initializing layer 1: ReLU(inplace)
    E:  1 'ReLU' object has no attribute 'weight'
initializing layer 2: MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    E:  2 'MaxPool2d' object has no attribute 'weight'
initializing layer 3: Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        HERE1:  3 torch.Size([99, 28, 3, 3])
        HERE2:  3 torch.Size([99, 28, 3, 3])
initializing layer 4: ReLU(inplace)
    E:  4 'ReLU' object has no attribute 'weight'
initializing layer 5: MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    E:  5 'MaxPool2d' object has no attribute 'weight'
initializing layer 6: Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        HERE1:  6 torch.Size([152, 99, 3, 3])
        HERE2:  6 torch.Size([152, 99, 3, 3])
initializin

In [62]:
x

VGG16_conv_rem(
  (features): Sequential(
    (0): Conv2d (3, 28, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d (28, 99, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d (99, 152, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d (152, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (11): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU()
    (15): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (16): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU()
    (18): Conv

In [47]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)


In [63]:
save_checkpoint({
        'epoch': 1,
        'arch': 'vgg11_little_rem_filt',
        'state_dict': x.state_dict(),
        'best_prec1': 0,
    }, "vgg11_little_rem_filt.pth.tar")